In [ ]:
# [YOUR_PROJECT_ID]を自分のGCPプロジェクトIDに置換してください。
PROJECT_ID="[YOUR_PROJECT_ID]"

BUCKET_NAME="{0}-mlengine".format(PROJECT_ID)
REGION="us-central1"

SAVED_MODEL_DIR="saved_model"
LATEST_MODEL_DIR=!(ls ./$SAVED_MODEL_DIR | tail -1)
LATEST_MODEL_DIR=LATEST_MODEL_DIR[0]

MODEL_DIR="gs://" + BUCKET_NAME + "/" + LATEST_MODEL_DIR
MODEL_NAME="news_classification"
VERSION_NAME="v2"
FRAMEWORK="TENSORFLOW"
MACHINE_TYPE="mls1-c1-m2"

In [ ]:
# エクスポートしたモデルを確認
! saved_model_cli show --all --dir $SAVED_MODEL_DIR/$LATEST_MODEL_DIR

In [ ]:
# Cloud Storage バケットの作成
! gsutil mb -l $REGION gs://$BUCKET_NAME

In [ ]:
# エクスポートしたモデルを Cloud Storage にアップロードする
! gsutil cp -r "$SAVED_MODEL_DIR/$LATEST_MODEL_DIR" $MODEL_DIR

In [ ]:
# ローカル予測でモデルをテストする（JSON）
! gcloud ai-platform local predict --model-dir $MODEL_DIR \
  --json-instances input.json \
  --framework tensorflow

In [ ]:
# モデルの作成
! gcloud ai-platform models create $MODEL_NAME

In [ ]:
# モデルのバージョンを作成
! gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin $MODEL_DIR \
  --runtime-version=1.14 \
  --framework $FRAMEWORK \
  --python-version=3.5 \
  --machine-type $MACHINE_TYPE

In [ ]:
# 新しいバージョンに関する情報を取得
! gcloud ai-platform versions describe $VERSION_NAME \
  --model $MODEL_NAME

In [ ]:
# 予測のリクエストのテスト
! gcloud ai-platform predict \
  --model $MODEL_NAME \
  --version $VERSION_NAME \
  --json-instances input.json